# Testing statistical signficance of ACS changes

#### Part 1: Code

Writing code to test statistical significance as described here:

* https://www2.census.gov/programs-surveys/acs/tech_docs/statistical_testing/2017StatisticalTesting5year.pdf

#### Part 2: Testing output of code:

Testing the results against this Excel sheet testing tool:

* https://www.census.gov/programs-surveys/acs/guidance/statistical-testing-tool.html

# Part 1: Code

In [11]:
class Estimate:
    
    def __init__(self, value, moe, Z=1.645):
        self.value = float(value)
        self.moe = float(moe)
        self.se = float(self.moe) / Z
        self.max = self.value + abs(self.moe)
        self.min = self.value - abs(self.moe)
        self.ci = (self.min, self.max)
        
        """ Standard Error = Margin of Error / Z

        where Z = 1.645 for 2006 and later years as well as all multiyear 
        estimates and Z = 1.65 for 2005 and earlier years."""
        
        
A = Estimate(10,1)
B = Estimate(11,1)
print (A.se)
print (A.ci)

0.607902735562
(9.0, 11.0)


True


In [44]:
class Proportion(object):
    
    def calculate_se(self):
        if self.P == 1:
            return self.A.se / self.se.B.value
        
        coef = 1 / self.B.value
        
        radicand = np.power(self.A.se, 2) - np.power(self.P, 2) * np.power(self.B.se, 2)
        
        if radicand < 0:
            radicand = np.power(self.A.se, 2) + np.power(self.P, 2) * np.power(self.B.se, 2)
            
        return coef * radicand

    
    def __init__(self, A, B):
        self.A = A
        self.B = A
        self.P = A.value / B.value
        self.se = self.calculate_se()
        self.value = self.P

p = Proportion(
    Estimate(10,1),
    Estimate(100,1)
)

p.se

0.03658502785450984

In [45]:
class Percentage (Proportion):
    
    def calculate_se(self):
        return super(Percentage, self).calculate_se() * 100

In [46]:
p2 = Percentage(
    Estimate(10,1),
    Estimate(100,1)
)

p2.se

3.6585027854509837

In [47]:
import numpy as np
class Difference:
    
    def Z(self):
        num = (self.A.value - self.B.value) 
        denom = np.sqrt(
            np.power(self.A.se, 2) +\
            np.power(self.B.se, 2)
        )
        
        return num / denom
    
    def is_significant(self, thresh=1.645):
        return abs(self.Z()) > 1.645
    
    def __init__(self, A, B):
        self.A = A
        self.B = B
        
print Difference(A, B).Z()
print Difference(A, B).is_significant()
print Difference(
    Estimate(10,1), 
    Estimate(12,1)
).is_significant()

-1.1631906550518707
False
True


In [50]:
pdiff = Difference(
    Proportion(
        Estimate(4567, 432),
        Estimate(350123456,120000)),
    Proportion(
        Estimate(5678,543),
        Estimate(351123456,121000)
    )
)

print pdiff.Z()
print pdiff.is_significant()

-1.2805483798954937e-07
False


In [ ]:
# Test against a spreadsheet
#22147046	+/-234,312	
#17997953	+/-199,112
#STD ERRORS: 
#   142438.91
#   121,040.73
#ZSCORE

In [51]:
Estimate(22147046,234312).se

142438.90577507598

In [53]:
Estimate(17997953,199112).se

121040.72948328267

In [55]:
Difference(
    Estimate(22147046,234312),
    Estimate(17997953,199112)
).Z()

22.196964278935535

In [60]:
Difference(
    Estimate(91,1),
    Estimate(93,0.9)
).is_significant()

True

In [64]:
import pandas as pd 
df = pd.read_csv("./data/vacant.csv").set_index("town")
df.columns = "vacant1","moe1","vacant2","moe2"

In [65]:
df.head()

,vacant1,moe1,vacant2,moe2
town,,,,
Bethel,3.6,1.5,5.8,2.3
Bridgeport,13.1,0.9,13.4,0.9
Brookfield,8.6,3.1,6.9,3.0
Danbury,9.3,1.2,9.1,1.1
Darien,5.6,2.0,4.9,1.7


In [71]:
df["sig_diff"] = df.apply(lambda x: Difference(
    Estimate(x["vacant1"],x["moe1"]),
    Estimate(x["vacant2"],x["moe2"])
).is_significant(), axis=1)

In [79]:
df["z"] = df.apply(lambda x: Difference(
    Estimate(x["vacant1"],x["moe1"]),
    Estimate(x["vacant2"],x["moe2"])
).Z(), axis=1)

In [85]:
df[df["sig_diff"]].head()

,vacant1,moe1,vacant2,moe2,sig_diff,z
town,,,,,,
Monroe,1.7,1.4,6.1,2.3,True,-2.688126
New Fairfield,13.2,2.7,17.4,2.5,True,-1.877612
Stamford,6.6,0.9,8.7,1.0,True,-2.567710
Trumbull,2.2,1.2,4.0,1.2,True,-1.744786
Avon,2.5,1.5,5.3,2.2,True,-1.729820


In [105]:
# Test against this spreadsheet, generated by 
# the Census significance tester

check = pd.read_csv("data/results.tsv",sep="\t").set_index("Label")
check["sig_diff_check"] = check["Statistically Different?"] == "Yes"
combined = check.join(df)
print (len(combined[combined["sig_diff_check"] != combined["sig_diff"]]))

0


In [ ]:
combined["Z-score"]

In [114]:
combined["zdiff"] = combined.apply(lambda x: max(
    x["z"],
    x["Z-score"]
) - max(
    x["z"],x["Z-score"]
), axis=1)

print (len(combined[combined["zdiff"] > 0]))

0


In [119]:
combined["SEA"] = combined.apply(
    lambda x: Estimate(x["vacant1"],
                       x["moe1"]).se,
    axis=1
)
combined["SEB"] = combined.apply(
    lambda x: Estimate(x["vacant2"],
                       x["moe2"]).se,
    axis=1
)
combined[["SEB","Second SE"]].head()

,SEB,Second SE
Label,,
Bethel,1.398176,1.40
Bridgeport,0.547112,0.55
Brookfield,1.823708,1.82
Danbury,0.668693,0.67
Darien,1.033435,1.03


In [123]:
# Those look pretty good


In [124]:
combined[["SEA","First SE"]].tail()

,SEA,First SE
Label,,
Scotland,2.492401,2.49
Sterling,3.525836,3.53
Thompson,2.066869,2.07
Windham,1.458967,1.46
Woodstock,2.431611,2.43
